# Convolution neural network

In [1]:
import pandas as pd, numpy as np, os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # settings like template by default

from dotenv import load_dotenv
from pathlib import Path

load_dotenv(dotenv_path = Path('.') / '.config')

pio.templates.default = os.getenv("template_plotly") # default template

import tensorflow as tf

only_cpu = True
if only_cpu: 
    os.environ["CUDA_VISIBLE_DEVICES"] = str(os.getenv("CUDA_VISIBLE_DEVICES"))
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(os.getenv("TF_CPP_MIN_LOG_LEVEL"))  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR
else:
    gpus = tf.config.list_physical_devices('GPU')  # obtiene las gpus instaladas
    if len(gpus) > 0:   # If GPU available
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(gpus)

2025-05-28 09:31:16.190328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748424676.204477   13904 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748424676.208728   13904 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748424676.221161   13904 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748424676.221172   13904 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748424676.221174   13904 computation_placer.cc:177] computation placer alr

In [4]:
# Read data
df = pd.read_csv('data/airline-passengers.csv', header=0, index_col=0, parse_dates=True)
print(f"Size data: {df.shape} min date: {min(df.index)} and max date: {max(df.index)}")
df.head(3)

Size data: (144, 1) min date: 1949-01-01 00:00:00 and max date: 1960-12-01 00:00:00


,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132


In [5]:
train = df.loc[:'1959-12-01']
test = df.loc['1959-01-01':] # test set starts before the end of the train set
print(f"Train set: {train.shape} and Test set: {test.shape}")
print(f"Train maz date: {max(train.index)} and test min date {min(test.index)}")

Train set: (132, 1) and Test set: (24, 1)
Train maz date: 1959-12-01 00:00:00 and test min date 1959-01-01 00:00:00


In [6]:
raw_seq = train.values.flatten()
print(f"Raw sequence length: {len(raw_seq)}")
print(raw_seq)

Raw sequence length: 132
[112 118 132 129 121 135 148 148 136 119 104 118 115 126 141 135 125 149
 170 170 158 133 114 140 145 150 178 163 172 178 199 199 184 162 146 166
 171 180 193 181 183 218 230 242 209 191 172 194 196 196 236 235 229 243
 264 272 237 211 180 201 204 188 235 227 234 264 302 293 259 229 203 229
 242 233 267 269 270 315 364 347 312 274 237 278 284 277 317 313 318 374
 413 405 355 306 271 306 315 301 356 348 355 422 465 467 404 347 305 336
 340 318 362 348 363 435 491 505 404 359 310 337 360 342 406 396 420 472
 548 559 463 407 362 405]


## Univariate series, one output (One step)

In [7]:
# rolling window from univariate time series
def split_sequence(sequence: list, n_steps: int, n_features: int):
    X, y = list(), list()
    for i in range(len(sequence)):
        # calculate sequence end index
        end_ix = i + n_steps
        # if index is greater than the length of the sequence, stop generating samples
        if end_ix > len(sequence)-1:
            break
        
        X.append( sequence[i:end_ix] ) # add the current sequence slice to the input features
        y.append( sequence[end_ix] ) # add the next value as the target variable
    
    X = np.array(X)
    # reshape from [samples, timesteps] to [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    y = np.array(y)
    return X, y

# number of steps
n_steps = 12
n_features = 1

# Generate train sample
X, y = split_sequence(raw_seq, n_steps, n_features)
print(X.shape, y.shape)

# Show the first sample
for i in range(1):
    print(X[i].tolist(), y[i])

(120, 12, 1) (120,)
[[112], [118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118]] 115


In [8]:
%%time
# Define a CNN model
model1 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_steps, n_features)),
    tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu'), # 64 filtros de tamaño 2
    tf.keras.layers.MaxPooling1D(pool_size=2), # maxpooling size = 2
    tf.keras.layers.Flatten(), # flaten layer
    tf.keras.layers.Dense(50, activation='relu'), # dense layer
    tf.keras.layers.Dense(1) # one output neuron
])

model1.compile(optimizer='adam', loss='mse')
# Train data
model1.fit(X, y, epochs=100, verbose=False)
# modelo.summary()

CPU times: user 4.43 s, sys: 1.44 s, total: 5.87 s
Wall time: 4.58 s


In [9]:
losses_df1 = pd.DataFrame(model1.history.history)
fig = px.line(losses_df1, x = losses_df1.index, y='loss',
            title='Loss in CNN model with one step ')
fig.show()

In [10]:
# Generate test sample
X_test, y_test = split_sequence(test.values.flatten(), n_steps, n_features)
print(X_test.shape, y_test.shape)
for i, val in enumerate(X_test):
    print(f"Test sample {i+1}: {val.tolist()} -> Target: {y_test[i]}")

(12, 12, 1) (12,)
Test sample 1: [[360], [342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405]] -> Target: 417
Test sample 2: [[342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417]] -> Target: 391
Test sample 3: [[406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391]] -> Target: 419
Test sample 4: [[396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419]] -> Target: 461
Test sample 5: [[420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461]] -> Target: 472
Test sample 6: [[472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472]] -> Target: 535
Test sample 7: [[548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535]] -> Target: 622
Test sample 8: [[559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535], [622]] -> Target: 606
Test sample 9: [[463], [407], [362], [405], [417], [391], [419

In [11]:
updated_array = X_test[0:1, :, :]
list_predicted = []
for index in range(X_test.shape[0]):
    yhat = model1.predict(updated_array, verbose=False)  # predice el punto siguiente
    list_predicted.append(yhat[0, 0].tolist())
    print(f"Predicted value: {yhat.shape} {yhat.tolist()}")
    updated_array = np.concatenate((updated_array[:, 1:, :], yhat.reshape(1, 1, 1)), axis=1)

Predicted value: (1, 1) [[384.63427734375]]
Predicted value: (1, 1) [[400.99224853515625]]
Predicted value: (1, 1) [[410.1405029296875]]
Predicted value: (1, 1) [[438.3272705078125]]
Predicted value: (1, 1) [[494.4099426269531]]
Predicted value: (1, 1) [[550.8048706054688]]
Predicted value: (1, 1) [[567.7857055664062]]
Predicted value: (1, 1) [[549.37158203125]]
Predicted value: (1, 1) [[492.2484130859375]]
Predicted value: (1, 1) [[442.69329833984375]]
Predicted value: (1, 1) [[440.3246154785156]]
Predicted value: (1, 1) [[425.8229675292969]]


In [12]:
print(list_predicted)
print(y_test.shape)
print(y_test)
# score = np.sqrt(mean_squared_error(list_predicted, y_test))
rmse = np.sqrt(np.mean((y_test - np.array(list_predicted))**2))
print("RMSE:", rmse)

[384.63427734375, 400.99224853515625, 410.1405029296875, 438.3272705078125, 494.4099426269531, 550.8048706054688, 567.7857055664062, 549.37158203125, 492.2484130859375, 442.69329833984375, 440.3246154785156, 425.8229675292969]
(12,)
[417 391 419 461 472 535 622 606 508 461 390 432]
RMSE: 31.347336090938416


In [13]:
compared_df = test['1960-01-01':].copy()
compared_df['Predicted'] = list_predicted
print(compared_df.head(2))
rmse = np.sqrt(np.mean((compared_df.Passengers - compared_df.Predicted)**2))
fig = px.line(compared_df, x = compared_df.index, y= ['Passengers', 'Predicted'],
            labels = {'value': 'Number of passengers', 'variable': 'Legend'},
            title = f'Prediction with a CNN model (one step) RMSE: {rmse}')
fig.show()

            Passengers   Predicted
Month                             
1960-01-01         417  384.634277
1960-02-01         391  400.992249


## Univariate series, one output (Multi step)

In [12]:
def split_sequence(sequence: list, n_steps_in: int, n_steps_out: int, n_features: int):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_in 
        out_end_ix = end_ix + n_steps_out
        
        if out_end_ix > len(sequence):
            break
        
        X.append( sequence[i:end_ix] ) 
        y.append( sequence[end_ix:out_end_ix] )
    
    X = np.array(X)
    # reshape from [samples, timesteps] to [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    y = np.array(y)
    return X, y

# choose the number of steps for input and output
n_steps_in, n_steps_out, n_features = 12, 3, 1

X, y = split_sequence(raw_seq, n_steps_in, n_steps_out, n_features)
print(X.shape, y.shape)
# show the shape of the generated samples
for i in range(2):
    print(X[i].tolist(), y[i])

(118, 12, 1) (118, 3)
[[112], [118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118]] [115 126 141]
[[118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118], [115]] [126 141 135]


In [13]:
# Define a CNN model
model2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_steps_in, n_features)),
    tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu'), # 64 filtros de tamaño 2
    tf.keras.layers.MaxPooling1D(pool_size=2), # maxpooling size = 2
    tf.keras.layers.Flatten(), # flaten layer
    tf.keras.layers.Dense(50, activation='relu'), # dense layer
    tf.keras.layers.Dense(n_steps_out) # one output neuron
])

model2.compile(optimizer='adam', loss='mse')
model2.fit(X, y, epochs=100, verbose=False)

In [22]:
losses_df2 = pd.DataFrame(model2.history.history)
fig = px.line(losses_df2, x = losses_df1.index, y='loss', 
            title='Loss in CNN model with multiple steps')
fig.show()

In [14]:
# Generate test sample
X_test, y_test = split_sequence(test.values.flatten(), n_steps_in, n_steps_out, n_features)
print(X_test.shape, y_test.shape)
for i, val in enumerate(X_test):
    print(f"{i+1}: {val.tolist()} -> Target: {y_test[i]}")

(10, 12, 1) (10, 3)
1: [[360], [342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405]] -> Target: [417 391 419]
2: [[342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417]] -> Target: [391 419 461]
3: [[406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391]] -> Target: [419 461 472]
4: [[396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419]] -> Target: [461 472 535]
5: [[420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461]] -> Target: [472 535 622]
6: [[472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472]] -> Target: [535 622 606]
7: [[548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535]] -> Target: [622 606 508]
8: [[559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535], [622]] -> Target: [606 508 461]
9: [[463], [407], [362], [405], [417], [391], [419], [461], [472], [535], [6

In [15]:
updated_array = X_test[0:1, :, :]
yhat = model2.predict(updated_array, verbose=False)
yhat

array([[401.12863, 414.6392 , 430.98096]], dtype=float32)

In [16]:
updated_array[:, 3:, :].shape, yhat.shape

((1, 9, 1), (1, 3))

In [17]:
updated_array = X_test[0:1, :, :]
list_predicted = []
limit = X_test.shape[0]
for index in range(limit):
    yhat = model2.predict(updated_array, verbose=False)  # predice el punto siguiente
    if index == limit - 1:
        my_end = yhat.flatten().tolist()
        list_predicted.extend(my_end)
        print("end predicted value:", my_end)
    else:
        list_predicted.append(yhat[0, 0].tolist())
    print(f"Predicted value: {yhat.shape} {yhat.tolist()}")
    updated_array = np.concatenate((updated_array[:, 3:, :], yhat.reshape(1, 3, 1)), axis=1)
print(f"length of list_predicted: {len(list_predicted)}")
print(list_predicted)

Predicted value: (1, 3) [[401.1286315917969, 414.6391906738281, 430.98095703125]]
Predicted value: (1, 3) [[457.23675537109375, 497.4270935058594, 541.303466796875]]
Predicted value: (1, 3) [[601.6732788085938, 557.5345458984375, 525.9202880859375]]
Predicted value: (1, 3) [[463.5365295410156, 444.7645568847656, 431.4413146972656]]
Predicted value: (1, 3) [[440.5992126464844, 452.9763488769531, 474.5220642089844]]
Predicted value: (1, 3) [[524.6293334960938, 570.1502075195312, 607.0521240234375]]
Predicted value: (1, 3) [[628.9524536132812, 591.3245849609375, 564.904296875]]
Predicted value: (1, 3) [[510.6441345214844, 485.9542541503906, 474.1505432128906]]
Predicted value: (1, 3) [[486.0848693847656, 505.45086669921875, 533.08349609375]]
end predicted value: [589.1981811523438, 626.6797485351562, 660.1609497070312]
Predicted value: (1, 3) [[589.1981811523438, 626.6797485351562, 660.1609497070312]]
length of list_predicted: 12
[401.1286315917969, 457.23675537109375, 601.6732788085938, 

In [18]:
compared2_df = test['1960-01-01':].copy()
compared2_df['Predicted'] = list_predicted
compared2_df.head(2)

,Passengers,Predicted
Month,,
1960-01-01,417,401.128632
1960-02-01,391,457.236755


In [19]:
# np.array(list_predicted).shape, y_test[:, 0].shape
rmse = np.sqrt(np.mean((compared2_df.Passengers - compared2_df.Predicted)**2))
print("RMSE:", rmse)

RMSE: 120.15253349774555


In [20]:
fig = px.line(compared2_df, x = compared2_df.index, y= ['Passengers', 'Predicted'],
            labels={'value': 'Number of passengers', 'variable': 'Legend'},
            title='Prediction with a CNN model (multi-step)')
fig.show()